<a href="https://colab.research.google.com/github/hamednasr/tensorflow/blob/main/06_transfer_learning_in_tensorflow_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 import tensorflow as tf

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-03-08 07:22:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-03-08 07:22:51 (50.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import compare_historys, create_tensorboard_callback,plot_loss_curves,unzip_data,walk_through_dir 

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2022-03-08 07:22:51--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.217.128, 173.194.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   186MB/s    in 7.9s    

2022-03-08 07:22:59 (197 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [6]:
unzip_data('101_food_classes_10_percent.zip')

In [7]:
train_data_10percent = tf.keras.preprocessing.image_dataset_from_directory\
('/content/101_food_classes_10_percent/train',
 label_mode='categorical',
 image_size=(224,224))

Found 7575 files belonging to 101 classes.


In [8]:
test_data = tf.keras.preprocessing.image_dataset_from_directory\
('/content/101_food_classes_10_percent/test',
 label_mode='categorical',
 image_size=(224,224),shuffle=False)

Found 25250 files belonging to 101 classes.


In [9]:
walk_through_dir('101_food_classes_10_percent')

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/grilled_cheese_sandwich'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/baklava'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/gyoza'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/mussels'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/spaghetti_carbonara'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/pork_chop'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/french_toast'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/chocolate_mousse'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/bibimbap'.
There are 0 directories and 250 im

In [10]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model101_10percent_checkpoint',
                                                save_weights_only=True,
                                                monitor='val_accuracy',
                                                save_best_only=True)

In [11]:
data_augmentation = tf.keras.Sequential([
                    tf.keras.layers.RandomFlip('horizontal'),
                    tf.keras.layers.RandomRotation(0.2),
                    tf.keras.layers.RandomZoom(0.2),
                    tf.keras.layers.RandomHeight(0.2),
                    tf.keras.layers.RandomWidth(0.2)], 
                    name ="data_augmentation")

In [28]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False,pooling='avg')
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224,224,3),name='input_layer')

x = data_augmentation(inputs)

x = base_model(x)
print(x.shape)
outputs = tf.keras.layers.Dense(101,activation='softmax',name='output_layer')(x)

model_101FE = tf.keras.Model(inputs,outputs,name='model_101FE')

model_101FE.compile(loss = 'categorical_crossentropy',
                  optimizer = tf.keras.optimizers.Adam(),
                  metrics = ['accuracy'])

model_101FE.summary()

(None, 1280)
Model: "model_101FE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [29]:
len(model_101FE.trainable_variables)

2

In [30]:
!rm -r '/content/tensorboard'

In [31]:
model_101FEcheck=tf.keras.callbacks.ModelCheckpoint(
                                    '/checkpoints/model_101FE.ckpt',
                                    save_weights_only=True,
                                    save_best_only=True,
                                    monitor='val_accuracy')

In [ ]:
history_model_101FE=model_101FE.fit(train_data_10percent,
                                     epochs=5,
                                     validation_data=test_data,
                                     validation_steps = int(0.15*len(test_data)),
                                     callbacks=[create_tensorboard_callback(dir_name='tensorboard',experiment_name='model_101FE'),
                                                model_101FEcheck])

Saving TensorBoard log files to: tensorboard/model_101FE/20220308-073535
Epoch 1/5
222/237 [===========================>..] - ETA: 4s - loss: 3.5559 - accuracy: 0.2394